In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [2]:
articles_df = pd.read_csv('teaable2500.csv')

articles_df

,차품목,효능번호,카페인,향기준,contentId,"맛,향",personId
0,Bengal Spice,1,X,2,100001,"계피, 생강",1
1,BerryBlossom White,1,O,1,100002,과일향(베리향),2
2,Black Cherry Berry,3,X,1,100003,"과일향(체리, 베리)",3
3,Breakfast in Paris Black Tea,1,O,"1,4",100004,"라벤더꽃향, 시트러스향",4
4,ButterScotch Blondie,3,O,4,100005,"버터카라멜향, 바닐라향",5
...,...,...,...,...,...,...,...
2495,aa2496,4,O,3,102496,190aa19,2496
2496,aa2497,4,O,4,102497,401aa71,2497
2497,aa2498,3,O,1,102498,262aa31,2498
2498,aa2499,2,X,4,102499,27aa27,2499


In [3]:
interaction_df = pd.read_csv('userinteract7.csv')
interaction_df.head(30)

,Unnamed: 0,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength,interactCnt,gender,age(15~60)
0,866,1463138398,VIEW,102336,-9.220000e+18,-4.480000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
1,1475,1463656314,VIEW,102124,-9.220000e+18,-7.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
2,2916,1464113091,VIEW,101361,-9.220000e+18,-2.770000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
3,5309,1462283851,VIEW,100580,-9.220000e+18,2.630000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
4,9221,1462452127,VIEW,102394,-9.220000e+18,-3.670000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
5,15868,1467140701,VIEW,100893,-9.220000e+18,1.270000e+18,NaN,NaN,NaN,1.0,43,male,18
6,15904,1467140700,VIEW,100577,-9.220000e+18,1.270000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
7,16744,1463055684,VIEW,102339,-9.220000e+18,-6.090000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
8,17230,1461322785,VIEW,102405,-9.220000e+18,6.210000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
9,17231,1461322815,VIEW,100498,-9.220000e+18,6.210000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18


In [4]:
interaction_df_over5 = (interaction_df
  .groupby('personId', group_keys=False)
  .apply(lambda df: df.assign(interactCnt = lambda d: d['contentId'].nunique()))
  .loc[lambda d: d['interactCnt'] >= 5]
)

In [5]:
event_type_strength = {
    'VIEW': 1.0,
    'JJIM': 3.0, 
    'comment1' : 1.0,
    'comment2' : 2.0,
    'comment3' : 3.0,
    'comment4' : 4.0,
    'comment5' : 5.0
  
}

interaction_df['eventStrength'] = (
  interaction_df
    .loc[:, 'eventType']
    .apply(lambda d: event_type_strength[d])
)

In [6]:
interaction_df_over5.shape[0]

69868

In [8]:
interaction_full_df = (
  interaction_df_over5
    .groupby(['personId', 'contentId'], as_index=False)['eventStrength']
    .sum()
    .assign(eventScore = lambda d: np.log2(1+d['eventStrength']))
)

interaction_full_df.head(10)

,personId,contentId,eventStrength,eventScore
0,-9.220000e+18,100050,1.0,1.0
1,-9.220000e+18,100205,1.0,1.0
2,-9.220000e+18,100214,1.0,1.0
3,-9.220000e+18,100332,1.0,1.0
4,-9.220000e+18,100380,1.0,1.0
5,-9.220000e+18,100416,1.0,1.0
6,-9.220000e+18,100427,1.0,1.0
7,-9.220000e+18,100433,1.0,1.0
8,-9.220000e+18,100487,1.0,1.0
9,-9.220000e+18,100498,1.0,1.0


In [9]:
interaction_train, interaction_test = train_test_split(
    interaction_full_df,
    stratify=interaction_full_df['personId'],
    test_size=0.2,
    random_state=42
)

In [10]:
interaction_full_indexed = interaction_full_df.set_index('personId')
interaction_train_indexed = interaction_train.set_index('personId')
interaction_test_indexed = interaction_test.set_index('personId')

In [11]:
def get_items_interacted(person_id, interaction_df):
    interated_items = interaction_df.loc[person_id]['contentId']
    return set(interated_items if type(interated_items) == pd.Series else [interated_items])

In [12]:
class ModelEvaluator:
    def __init__(self, n_non_interacted=100):
        self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = n_non_interacted
        
    def get_non_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interaction_full_indexed)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items
        
        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)
        
    def _verify_hit_top_n(self, item_id, recommend_items, topn):
        try:
            index = next(i for i, c in enumerate(recommend_items) if c == item_id)
        except:
            index = -1
        hit = int(index in range(0, topn))
        return hit, index
    
    def evaluate_model_for_user(self, model, person_id):
        interacted_values_testset = interaction_test_indexed.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])
        
        interacted_items_count_testset = len(person_interacted_items_testset)
        
        # 특정 사용자에 대한 추천 순위 목록을 받아온다
        person_recs = model.recommend_items(
            person_id,
            items_to_ignore=get_items_interacted(person_id, interaction_train_indexed),
            topn=10000000000
        )
        
        hits_at_5_count = 0
        hits_at_10_count = 0
        
        # test셋에서 사용자가 상호작용한 모든 항목에 대해 반복한다
        for item_id in person_interacted_items_testset:
            
            # 사용자가 상호작용하지 않은 100개 항목을 샘플링한다
            non_interacted_items_sample = self.get_non_interacted_items_sample(
                person_id,
                sample_size=self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS,
                seed=item_id % (2**32)
            )
            
            # 현재 선택한 item_id(상호작용 있었던 항목)와 100개 랜덤 샘플을 합친다
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))
            
            # 추천 결과물 중에서 현재 선택한 item_id와 100개 랜덤 샘플의 결과물로만 필터링한다
            valid_recs_df = person_recs[person_recs['contentId'].isin(items_to_filter_recs)]
            valid_recs = valid_recs_df['contentId'].values
            
            # 현재 선택한 item_id가 Top-N 추천 결과 안에 있는지 확인한다
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10
            
        # Recall 값은 상호작용 있었던 항목들 중에서 관련없는 항목들과 섞였을 때 Top-N에 오른 항목들의 비율로 나타낼 수 있다
        recall_at_5 = hits_at_5_count / interacted_items_count_testset
        recall_at_10 = hits_at_10_count / interacted_items_count_testset
        
        person_metrics = {
            'hits@5_count': hits_at_5_count,
            'hits@10_count': hits_at_10_count,
            'interacted_count': interacted_items_count_testset,
            'recall@5': recall_at_5,
            'recall@10': recall_at_10
        }
        return person_metrics
    
    def evaluate_model(self, model):
        people_metrics = []
        for idx, person_id in enumerate(list(interaction_test_indexed.index.unique().values)):
            person_metrics = self.evaluate_model_for_user(model, person_id)
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)

        print('{} users processed'.format(idx))
        
        detailed_result = (
            pd.DataFrame(people_metrics)
              .sort_values('interacted_count', ascending=False)
        )
        
        global_recall_at_5 = detailed_result['hits@5_count'].sum() / detailed_result['interacted_count'].sum()
        global_recall_at_10 = detailed_result['hits@10_count'].sum() / detailed_result['interacted_count'].sum()
        
        global_metrics = {
            'model_name': model.get_model_name(),
            'recall@5': global_recall_at_5,
            'recall@10': global_recall_at_10
        }
        
        return global_metrics, detailed_result
    
model_evaluator = ModelEvaluator(n_non_interacted=100)

In [13]:
interaction_full_df = (
  interaction_df_over5
    .groupby(['personId', 'contentId'], as_index=False)['eventStrength']
    .sum()
    .assign(eventScore = lambda d: np.log10(1+d['eventStrength']))
)

interaction_full_df.head(10)

,personId,contentId,eventStrength,eventScore
0,-9.220000e+18,100050,1.0,0.30103
1,-9.220000e+18,100205,1.0,0.30103
2,-9.220000e+18,100214,1.0,0.30103
3,-9.220000e+18,100332,1.0,0.30103
4,-9.220000e+18,100380,1.0,0.30103
5,-9.220000e+18,100416,1.0,0.30103
6,-9.220000e+18,100427,1.0,0.30103
7,-9.220000e+18,100433,1.0,0.30103
8,-9.220000e+18,100487,1.0,0.30103
9,-9.220000e+18,100498,1.0,0.30103


In [14]:
item_popularity = (interaction_full_df
 .groupby('contentId')['eventStrength'].sum()
 .sort_values(ascending=False)
 .reset_index()
)

item_popularity.head(10)

,contentId,eventStrength
0,101277,61.0
1,102020,60.0
2,100215,60.0
3,100731,59.5
4,101896,58.0
5,102115,57.5
6,100567,56.5
7,101079,56.0
8,101591,56.0
9,102257,55.5


In [15]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 인기상품 중에서 사용자가 보지 않았던 상품을 추천한다
        recommendations = (
          self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)]
            .sort_values('eventStrength', ascending=False)
            .head(topn)
        )
        
        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['eventStrength', 'contentId', 'title', 'url', 'lang']]
            )
            
        return recommendations

In [16]:
popularity_model = PopularityRecommender(item_popularity, articles_df)

In [17]:
print('Popularity 추천 모형을 평가합니다')
pop_global_metrics, pop_detailed_results = model_evaluator.evaluate_model(popularity_model)
print('Global Metrics:\n{}'.format(pop_global_metrics))
pop_detailed_results.head(10)


Popularity 추천 모형을 평가합니다
865 users processed
Global Metrics:
{'model_name': 'Popularity', 'recall@5': 0.06716132486804866, 'recall@10': 0.1301919987761034}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
137,16,35,264,0.060606,0.132576,-1.030000e+18
29,19,35,235,0.080851,0.148936,-1.440000e+18
174,10,24,221,0.045249,0.108597,3.610000e+18
76,14,25,177,0.079096,0.141243,-2.630000e+18
42,11,25,158,0.069620,0.158228,-3.600000e+18
18,14,22,149,0.093960,0.147651,-2.980000e+18
215,9,20,143,0.062937,0.139860,-7.090000e+17
4,7,16,143,0.048951,0.111888,3.300000e+18
8,7,10,127,0.055118,0.078740,3.640000e+18
85,3,11,123,0.024390,0.089431,1.120000e+18


In [ ]:
item_popularity.nunique()